 

# Allocate Minimum Number Of Pages Handout

In this problem, you are given an array where each element represents the number of pages in a book and a number of students. You must assign books to the students (keeping the order of the books intact) such that each student gets at least one book and the maximum number of pages allocated to any student is minimized.

---

## 1. IP–OP–PS (Input, Output, Problem Statement)

### Problem Statement
Given an array of integers (each integer representing the number of pages in a book) and an integer representing the number of students, determine the minimum possible value of the maximum number of pages assigned to a student. Books must be allocated in contiguous order.

**Key Operations**:
- **Contiguous Allocation**: Books must be assigned in the order given.
- **Minimize Maximum**: Among all possible valid allocations, choose the one where the maximum pages any student gets is minimized.
- **Return the Value**: If allocation is impossible (e.g., when the number of books is less than the number of students), return `-1`.

### Inputs
- **Array of Pages**: A list of positive integers, e.g., `[12, 34, 67, 90]`.
- **Number of Students**: A positive integer, e.g., `2`.

### Outputs
- **Minimum Maximum Pages**: A single integer representing the minimized maximum pages allocated to a student (e.g., `113`).

### Detailed Example

**Sample Input**:
```plaintext
Books: [12, 34, 67, 90]
Students: 2
```

**Sample Output**:
```plaintext
Minimum Maximum Pages: 113
```

*Explanation*:  
One optimal allocation is:
- Student 1: Books `[12, 34, 67]` → Total pages = 113  
- Student 2: Book `[90]` → Total pages = 90  
Here, the maximum pages assigned is 113, which is minimized among all valid allocations.

---

## 2. Identification

### Why Use Binary Search on Answer?

1. **Ordered Answer Range**:
   - The answer (i.e., the maximum pages a student gets) lies between the maximum number of pages in a single book (as no student can be allocated less than that) and the sum of all pages (if one student gets all books).

2. **Efficiency**:
   - A brute-force approach would check all possible allocations, which is exponential. Instead, we can binary search on the range of possible answers (from `max(pages)` to `sum(pages)`) to efficiently find the minimum feasible maximum.

3. **Feasibility Predicate**:
   - For any candidate maximum value, we can check if it’s possible to allocate books to all students such that no student receives more than that candidate value using a greedy approach.

---

## 3. Break Down → Two-Phase Approach

### Step-by-Step Sub-Tasks

1. **Define the Search Space**:
   - **Lower Bound**: The maximum number of pages in a single book (since each student must take at least one book).
   - **Upper Bound**: The total sum of pages (if one student gets all the books).

2. **Feasibility Function (Predicate)**:
   - Given a candidate value `mid`, simulate the allocation:
     - Iterate over the books, summing pages until adding another book would exceed `mid`.
     - Allocate the accumulated books to a student and start a new sum.
     - Count the number of students needed.
   - If the number of students required is less than or equal to the given number, then the allocation is feasible.

3. **Binary Search on Answer**:
   - While the lower bound is less than or equal to the upper bound:
     - Compute `mid = low + (high - low) / 2`.
     - If the allocation is feasible with `mid` as the maximum allowed pages, try to find a smaller value by setting `high = mid - 1`.
     - Otherwise, set `low = mid + 1`.
   - The optimal answer will be the smallest candidate for which the allocation is feasible.

4. **Return the Answer**:
   - After the binary search completes, the lower bound (`low`) will be the minimum maximum pages possible.

---

## 4. Explanations + Code

### Detailed Explanation

- **Search Space Initialization**:
  - `low` is set to the maximum pages in any single book.
  - `high` is set to the sum of all pages.
  
- **Feasibility Check**:
  - For a candidate maximum `mid`, iterate through the books and keep a running total. If adding the next book exceeds `mid`, assign the current set of books to one student and reset the total. Count how many students are needed.
  
- **Binary Search**:
  - Adjust the bounds based on the feasibility check until the minimal possible maximum is found.

### C++ Implementation

```cpp
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

// Helper function to check if it is possible to allocate books
// such that no student reads more than maxPages.
bool isFeasible(const vector<int>& pages, int students, int maxPages) {
    int requiredStudents = 1;  // Start with one student
    int currentSum = 0;
    
    for (int page : pages) {
        // If a single book has more pages than maxPages, allocation is impossible.
        if (page > maxPages)
            return false;
        
        if (currentSum + page > maxPages) {
            // Allocate current set to one student and reset sum for next student.
            requiredStudents++;
            currentSum = page;
            if (requiredStudents > students)
                return false;
        } else {
            currentSum += page;
        }
    }
    
    return true;
}

// Function to find the minimum possible maximum pages using binary search.
int allocateBooks(const vector<int>& pages, int students) {
    if (pages.size() < students)
        return -1;  // Not enough books to allocate to each student.
    
    int low = *max_element(pages.begin(), pages.end());
    int high = 0;
    for (int page : pages)
        high += page;
    
    int result = high;  // Initialize result with the highest possible value.
    
    while (low <= high) {
        int mid = low + (high - low) / 2;
        if (isFeasible(pages, students, mid)) {
            result = mid;  // Candidate is feasible.
            high = mid - 1;  // Try for a smaller maximum.
        } else {
            low = mid + 1;  // Increase the candidate value.
        }
    }
    
    return result;
}

int main() {
    vector<int> pages = {12, 34, 67, 90};
    int students = 2;
    
    int ans = allocateBooks(pages, students);
    cout << "Minimum Maximum pages: " << ans << endl;
    return 0;
}
```

**Explanation**:
- The function `isFeasible` checks if books can be allocated under the candidate maximum pages.
- The function `allocateBooks` applies binary search on the answer space defined by the maximum pages in a single book and the total pages.
- The final answer is the smallest value for which the allocation is possible.

---

## 5. Animated Visualization (Interactive Demo)

Below is a **Python** code snippet using `matplotlib` and `ipywidgets` to simulate the binary search process over the possible maximum page allocations. This interactive demo helps visualize how the search space is narrowed down.

```python
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import numpy as np

# Example books and number of students.
pages = [12, 34, 67, 90]
students = 2

def is_feasible(pages, students, max_pages):
    required_students = 1
    current_sum = 0
    for page in pages:
        if page > max_pages:
            return False
        if current_sum + page > max_pages:
            required_students += 1
            current_sum = page
            if required_students > students:
                return False
        else:
            current_sum += page
    return True

def binary_search_on_answer(pages, students):
    low = max(pages)
    high = sum(pages)
    steps = []
    
    while low <= high:
        mid = low + (high - low) // 2
        feasible = is_feasible(pages, students, mid)
        steps.append((low, mid, high, feasible))
        if feasible:
            high = mid - 1
        else:
            low = mid + 1
    return steps, low

steps, answer = binary_search_on_answer(pages, students)

def draw_step(step_idx=0):
    low, mid, high, feasible = steps[step_idx]
    
    plt.figure(figsize=(10, 2))
    title = f"Step {step_idx+1}: low = {low}, mid = {mid}, high = {high} | Feasible: {feasible}"
    plt.title(title)
    
    # Visualize the candidate range on a number line.
    candidates = np.arange(max(pages), sum(pages) + 1)
    values = np.zeros_like(candidates)
    plt.plot(candidates, values, 'ko', markersize=3)
    plt.plot(mid, 0, 'ro', markersize=10, label="Current Candidate (mid)")
    
    plt.xlabel('Maximum Pages Candidate')
    plt.yticks([])
    plt.legend()
    plt.xlim(max(pages) - 5, sum(pages) + 5)
    plt.show()

interact(draw_step, step_idx=IntSlider(min=0, max=len(steps)-1, step=1, value=0));
print("Minimum Maximum Pages Allocated =", answer)
```

**Visualization Explanation**:
- **Step Recording**:  
  The function `binary_search_on_answer` records each step with the current values of `low`, `mid`, and `high` along with whether the candidate is feasible.
- **Drawing Function**:  
  `draw_step` plots a number line showing the current candidate (`mid`) and displays the current search range.
- **Interactivity**:  
  Use the slider to step through each iteration of the binary search on the answer space.

---

## Final Notes

- **Problem Recap**:  
  The goal is to allocate books to students such that the maximum pages any student gets is minimized while preserving the contiguous order of books.
- **Algorithm Efficiency**:  
  Binary search on the answer space reduces the problem from an exponential search to O(log(sum of pages)) multiplied by the cost of the feasibility check.
- **Applications**:  
  This technique is commonly used in load balancing, resource allocation, and partitioning problems.
- **Visualization**:  
  The interactive demo provides clear insight into how the binary search iteratively narrows the candidate range.
 